In [2]:
import pandas as pd
from lyricsgenius import Genius
import string
import re
from collections import defaultdict


import os
import gensim
from gensim.utils import simple_preprocess
import gensim.downloader as api
import nltk
nltk.download('stopwords')
from gensim import corpora
from gensim.models import LdaModel, LdaMulticore
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import logging

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm
from pattern.en import lemma

C:\Users\user\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from gensim.models import CoherenceModel
from collections import Counter
from googletrans import Translator
translator = Translator()
import time
logging.basicConfig(format ='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level = logging.INFO)
 
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

# import stopwords
stop_words = stopwords.words('english')
# add stopwords
extended_stopwords = ["are", "im", "youre", "ill", "youll" "theyre" "hes", "shes", "oh", "ooh", "oohooh", "oohoohooh", "ay", "yo", "wo",
                      "wont", "might", "dont", "going", "even", 'one', 'think', 'feel', 'u',
                      "cant", 'yeah', 'yes', "no", "cause", "hey", "really", "said", "says", "saying",
                      "like", 'got', 'get', 'wanna', "wants", 'baby', 'know', "knows", 'need', "bout",
                      'feel', 'want', 'lets', ".", "ft", "feat", "lil", "g", "mc", "make", 
                      "give", "gimme", "uh", "uhuh", "come", "comes", "came", "coming", "some", 
                      "go", "aint", "take", "yeahyeah", "mm", "okay", "much", "more", "done", "did", 
                      "never", "ever", "always", "sometimes", "one", "going", "saying", "already", "even",
                      "yet", "wanted", "all", "bout", 'even', 'put', 'woah', "woo", 'anymore',
                      "see", "let", "lets", "day", "another", "other", "everybody", "would", "could",
                      "nobody", "somebody", "anybody", "gotta", "whats", "ahah", "couldnt", "ive", "id", "youve",
                      "nothing", "everything", "anything", "something", "knew", "tell", "say", "babe", "heres", "theres",
                      "uhh", "its", "itll", "also", "ohohoh", "gets", "getting", "without", "with", "gonna", "didnt", "thats", 
                      "ya", "every", "•", "ah", "ey", "thats", "ohoh", "tryna", "havent", "doesnt", "isnt", "whos", "whose", "still"]
stop_words = stop_words + extended_stopwords

C:\Users\user\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
def remove_chars(text):
    '''
    Input: text in str format
    Output: text with removed punctuation signs
    '''
    stopchars = string.punctuation
    return ''.join([c for c in text if c not in stopchars])

C:\Users\user\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
def translate_lyrics(lyrics):
    '''
    Input: text in str format
    Output: text translated to English
    '''
    lang = translator.detect(lyrics).lang
    if lang != "en":
        lyrics = translator.translate(lyrics, dest='en').text
    return {'translated_lyrics': lyrics, 'lang': lang}

C:\Users\user\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
def scrape_lyrics(file_path, num_songs=200, a=0, create_csv=True):
    '''
    Create an initial dataframe of the form: 
    #Pos	Artist and Title	Lyrics
    
    Args:
        file_path (str): accessed file, must be in excel (.xlsx) format
    
        num_songs (int): the number of songs need to be scraped
    
        a (int): the number of the song from which scraping should start 
    
        b (int): the number of the song at which scraping should end
    
        create_csv (bool): set as True to create a dataframe file based on the results of scraping
    '''
    
    # TODO: adjust the path
    country = file_path.strip('.xlsx').lower()
    
    file = pd.read_excel(file_path)
    file = file[a:]
    file = file[['Artist and Title']]
    artists = []
    titles = []
    
    for el in file.iloc[:, 0]:
        el = [word.strip() for word in el.split('-')]
        artists.append(el[0])
        titles.append(el[1])
        
    data = {'Artist': artists,
        'Title': titles}
    df = pd.DataFrame(data)
    new_df = pd.DataFrame()
    new_df['Artists'], new_df['Title'], new_df['Lyrics'] = None, None, None
    
    #Creating a separate column for lyrics
    df['Lyrics'] = None
    #number of songs
    n = len(df)
    
    token = "add-your-token-for-genius.com"

    genius = Genius(token)
    genius.verbose = False
    genius.remove_section_headers = True
    genius.timeout = 10
    genius.sleep = 10
    
    none_indices = []
    positive_indices = []
    
    j = a
    for i in tqdm(range(a, n)):

        try:
            artist = genius.search_artist(df.iloc[i,0], max_songs=3, sort="title")
            song = artist.song(df.iloc[i,1])
            df.iloc[i,2]=song.lyrics
            positive_indices.append(i)
            new_df.loc[j] = [df.iloc[i,0], df.iloc[i,1], df.iloc[i,2]]
            j += 1
            print(f"song number {i} : {df.iloc[i,1]} by {df.iloc[i,0]} was successfully scraped")
            if len(positive_indices) == num_songs:
                print(f"{num_songs} were successfully scraped")
                break
            
        except:
            print(f'error: missing lyrics for song number {i} : {df.iloc[i,1]} by {df.iloc[i,0]}')
            none_indices.append(i)
            
    if create_csv == True:    
        new_df.to_csv(f'df_{country}.csv')

    return new_df

C:\Users\user\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
class Country:
    '''
    The class to create and work with the country object.
    '''

    def __init__(self, country_name, num_songs=200):
        '''
        Initializes the instance of a Country class.

        Args:
        
          country_name (str): name of the country. Must start with capital letter.
          
          num_songs (int): the number of songs to create document collection from.

        '''
        self.country_name = country_name 
        self.num_songs = num_songs
        
        try:
            #if the dataframe file is already there, just read it 
            self.df = pd.read_csv(f"data/processed_dataframes/df_{country_name.lower()}.csv")
            #count how many lyrics were scraped
            self.count_indices()
            #if the number of scraped lyrics is the same as the number of songs reqiured, confirm success
            if len(self.present) == num_songs:
                print("The file was successfully read")
            #if for some reason the dataframe was empty, go to scraping the lyrics from scratch
            elif len(self.present) == 0:
                next
            #if the number of scraped lyrics was not 0, but less than needed, scrape more 
            elif 0 < len(self.present) < 200:
                self.scrape_more_lyrics(n=200, a=200)
                
                
        except FileNotFoundError:
            #if the dataframe file for the country wasn't created it, scrape the lyrics and create the df file.
            print("The dataframe will be created now. Starting scraping the lyrics")
            
            start = time.time()
            self.df = scrape_lyrics(f"data/song_charts/{country_name}.xlsx", num_songs=200)
            end = time.time()

            print("Done")
            print("Overall runtime = ", str(abs(start-end)))

    def count_indices(self):
        '''
        Create lists of the indices of missing and recalled lyrics. 
        Measure missing and recall rate.
        '''
        self.missing = []
        self.present = []
        for i in range(len(self.df['Lyrics'])):
            #print("Lyrics number", i)
            #print(str(self.df['Lyrics'][i]))
            if len(str(self.df['Lyrics'][i])) > 10:
                self.present.append(i)
            else:
                self.missing.append(i)
        
        self.missing_rate = len(self.missing)/self.num_songs*100
        self.recall_rate = len(self.present)/self.num_songs*100
        
        print(f"recall_rate: {self.recall_rate}%")
        
    
    def translate_all_lyrics(self):
        '''
        Translate the lyrics if they are not translated yet.
        Count the proportion of languages in the songs
        
        self.langs: shows languages used in the songs and their count.
        '''
        
        self.langs = []
    
        #Check if the Translations column is already created
        try:
            a = self.df['Translations'][self.present[-1]]
            if len(a) < 10:
                next
            
            print(f"Reading from df_{self.country_name.lower()}.csv file...")
            
            #Count languages used in the songs 
            for i in tqdm(range(len(self.df['Languages']))):
                lang = self.df['Languages'][i]
                if type(lang) == str:
                    self.langs.append(lang)
                elif type(lang) == list:
                    self.langs.append(tuple(lang))
                    
            print("Translations were successfully read.")

        #Create a column with translations and write it into CSV file
        except:
            
            try:
                
                print(f"Filling in the missing translations for the df_{self.country_name.lower()}.csv file...")
                
                for i in tqdm(range(self.last_translated, len(self.df['Lyrics']))):
                    lyrics = self.df['Lyrics'][i]
                    if len(str(lyrics)) > 10:
                        result = translate_lyrics(lyrics)
                        self.df['Translations'][i] = result['translated_lyrics']
                        self.df['Languages'][i] = result['lang']
                        lang = self.df['Languages'][i]

                        #if there is only one language in the song, add it as it is
                        if type(lang) == str:
                            self.langs.append(lang)

                        #if there is more than 1 language used, add as a tuple
                        elif type(lang) == list:
                            self.langs.append(tuple(lang))
                    else:
                        continue

                #update the dataframe to add translations
                self.df.to_csv(f'df_{self.country_name.lower()}.csv')
                
                
            except:
                
                print(f"Creating translations for the df_{self.country_name.lower()}.csv file...")
                
                try:
                    self.df['Translations'] = None
                    self.df['Languages'] = None

                    #Translate the lyrics using Google API
                    #Count languages used in the songs 

                    for i in tqdm(range(len(self.df['Lyrics']))):
                        self.translation_checkpoint = i 
                        lyrics = self.df['Lyrics'][i]
                        if len(str(lyrics)) > 10:
                            result = translate_lyrics(lyrics)
                            self.df['Translations'][i] = result['translated_lyrics']
                            self.df['Languages'][i] = result['lang']
                            lang = self.df['Languages'][i]

                            #if there is only one language in the song, add it as it is
                            if type(lang) == str:
                                self.langs.append(lang)

                            #if there is more than 1 language used, add as a tuple
                            elif type(lang) == list:
                                self.langs.append(tuple(lang))
                        else:
                            continue
                            
                except ReadTimeout:
                    print("The translation stopped at song", self.translation_checkpoint)
                    next

                #update the dataframe to add translations
                self.df.to_csv(f'df_{self.country_name.lower()}.csv')
            
            print("File with translations was created")
                         
                        
            #TODO: manage the error when Internet connection breaks and Google translator throws an error     
                    
            
        self.langs = dict(Counter(self.langs))
        
                
    def count_langs(self):
        '''
        To use if for some reason the self.langs variable value was lost 
        (usually after scraping new lyrics).
        '''
        self.langs = []
        for i in tqdm(range(len(self.df['Languages']))):
            lang = self.df['Languages'][i]
            if type(lang) == str:
                self.langs.append(lang)
            elif type(lang) == list:
                self.langs.append(tuple(lang))
                
        self.langs = dict(Counter(self.langs))
        
        print(f"Here is the distribution of languages in songs for{self.country_name}:", self.langs)
    
    def preprocess_lyrics(self, force_preprocess=False, delete_repetitions=False):
        '''
        Preprocesses the lyrics.

        - removes punctuation
        - splits sentences into words
        - removes extra words
        - removes words shorter than 3 letters (usually exclamations - "ay", "yo", etc.)
        
        If the lyrics are already preprocessed, just reads the dataframe file.

        Args:
        
          force_preprocess: boolean, set as True if you need to preprocess from scratch
          (usually if you added some changes into preprocessing steps and need to update
          the dataframe).
          
          delete_repetitions: boolean, set as True if you want to avoid repeating words 
          in the lyrics of each song.
          Recommended when lyrics are repetetive and inflate some words' count.
          
        
        '''    
        pattern_order = r'[0-9]'
        
        #if the lyrics are already processed and not to be updated, read from file.
        #if trying to access the last processed translation throws a KeyError, process from scratch. 
        
        if force_preprocess==False:
            a = self.df['Processed translations'][self.present[-1]]
            print(f"The lyrics are already processed and translated in df_{self.country_name.lower()}.csv")
            
        else:
            
            print("Processing translations...")
            
            self.df['Processed translations'] = None

            for i in self.present:
                try:
                    #delete headers, extra text
                    try: 
                        self.df['Processed translations'][i] = self.df['Translations'][i].split('Lyrics'.casefold())[1]
                    except IndexError:
                        self.df['Processed translations'][i] = self.df['Translations'][i]

                    #delete punctuation, split into words
                    self.df['Processed translations'][i] = remove_chars(self.df['Processed translations'][i]).split()
                    
                    #delete the word "Embed" in the end, if it is present
                    
                    try:
                        self.df['Processed translations'][i][-1] = re.sub(pattern_order, '', self.df['Processed translations'][i][-1].split('Embed')[0])
                    except IndexError:
                        next
                        
                    #make lowercase, lemmatize
                    #delete if the word is a stop word and/or if it is shorter than 3 letter
                    
                    if delete_repetitions==True:
                        lyrics = set()
                        for word in self.df['Processed translations'][i]:
                            word = lemmatizer.lemmatize(word.lower())
                            if word not in stop_words and len(word) > 2:
                                lyrics.add(word)
                        
                    else:
                        lyrics = []
                        for word in self.df['Processed translations'][i]:
                            word = lemmatizer.lemmatize(word.lower())
                            if word not in stop_words and len(word) > 2:
                                lyrics.append(word)
                            
                    self.df['Processed translations'][i] = lyrics

                except AttributeError:
                    next
                    
            #Add preprocessed lyrics to the dataframe
            self.df.to_csv(f'df_{self.country_name.lower()}.csv')
            
            print(f"Translations are processed and are available in df_{self.country_name.lower()}.csv")


    def return_songs(self):
        '''
        Returns a list of all available processed lyrics
        
        self.processed_songs: list of lists of words. Each list corresponds to a song.
        '''
        
        #else - just read words from the list
        
        self.processed_songs = []
        index = self.present[0]
                
        #if the dataframe file was read in the form of a string, parse it into words.
        #for example, type("['hello', 'world']") = str
        
        if type(self.df['Processed translations'][index]) is type('str'):

            for i in self.present:
                current_lyrics = []
                text = self.df['Processed translations'][i]
                text = text.split()
                for word in text:
                    current_lyrics.append(word.strip("',[]!?-.#@"))
                self.processed_songs.append(current_lyrics)
                    
        else:
            
            for i in self.present:
                self.processed_songs.append(self.df['Processed translations'][i])
        
        #Filter the lyrics which are too short (less than 10 words)
        self.processed_songs = list(filter(lambda x: (len(x) > 10), self.processed_songs))
    
    
    def return_songs_to_sentences(self):
        '''
        Divide songs into parts made of 4 lines.
        In this case, 1 document in collection = 4 lines from a song.
        '''   
        
        print("Dividing songs into 4-lined parts...")

        self.divided_songs = []

        for i in tqdm(self.present):
            lyrics = self.df['Translations'][i]
            
            #delete headers, extra text
            try: 
                lyrics = self.df['Translations'][i].split('Lyrics'.casefold())[1]
            except IndexError:
                self.df['Processed translations'][i] = self.df['Translations'][i]

            #delete punctuation, 
            #split into lines
            lyrics = remove_chars(lyrics).split("\n")
            for j in tqdm(range(0, len(lyrics), 4)):
                self.divided_songs.append(lyrics[j:j+4])


        for i in tqdm(range(len(self.divided_songs))):
            united_doc = []
            for line in self.divided_songs[i]:
                new_line = [word.lower() for word in line.split() if word.casefold() not in stop_words]
                united_doc.extend(new_line)
            self.divided_songs[i] = united_doc
            
            
    def find_topics(self, div_songs=False, filter_extremes=True, topics=10, epochs=50):
        '''
        Detect topics using gensim.models.LdaMulticore model
        
        div_songs: boolean, set as True to use songs divided into 4 lines sentences as input
        
        filter_extremes: boolean, filters the most and the least frequent words in the overall corpus
        
        topic: integer, number of topics you expect to see. Recommended: from 5 to 10.
        
        epochs: integer, number of epochs. 
        
        Visualize with pyLDAvis.
        '''
        
        if div_songs == False:
            self.return_songs()
            songs = self.processed_songs
        else:
            self.return_songs_to_sentences()
            songs = self.divided_songs
        
        country_dict = corpora.Dictionary(songs)
        
        if filter_extremes == True:
            #In this case, each word should be found in no less than 10 documents, 
            #and no more than 50% of the documents. 
            #Only top 15 000 unique tokens from the corpus will be saved.
            country_dict.filter_extremes(no_below=3, no_above=0.5, keep_n=15000)
            
        Corpus = [country_dict.doc2bow(l) for l in songs]

        print(f"Model training started. Parameters: {topics} topics, {epochs} epochs")

        LDA_model = gensim.models.LdaMulticore(corpus=Corpus,
                                           id2word=country_dict,
                                           num_topics=topics,
                                           passes=epochs)

        #Compute Coherence Score
        #Evaluation metric: u_mass (the closer to 0, the better).
        coherence_model_lda = CoherenceModel(model=LDA_model, texts=songs, dictionary=country_dict, coherence='u_mass')
        self.coherence_lda = coherence_model_lda.get_coherence()
        print('\nCoherence Score: ', self.coherence_lda)
        doc_lda = LDA_model[Corpus]

        pyLDAvis.enable_notebook()
        lda_viz = gensimvis.prepare(LDA_model, Corpus, country_dict)
        self.corpus = Corpus
        self.model = LDA_model
        self.topic_words = doc_lda
        self.visualize = lda_viz

        
    def find_topics_2(self, topics=5, filter_extremes=True, div_songs=False, epochs=100):
        
        if div_songs == False:
            self.return_songs()
            songs = self.processed_songs
        else:
            self.return_songs_to_sentences()
            songs = self.divided_songs
            
        country_dict = corpora.Dictionary(songs)
        
        if filter_extremes == True:
            #In this case, each word should be found in no less than 10 documents, 
            #and no more than 50% of the documents. 
            #Only top 15 000 unique tokens from the corpus will be saved.
            country_dict.filter_extremes(no_below=10, no_above=0.5, keep_n=15000)
            
        Corpus = [country_dict.doc2bow(l) for l in songs]
        start = time.time()
        LDA_model = LdaModel(corpus = Corpus, id2word=country_dict, num_topics = topics, update_every=1, chunksize=10, 
                             passes=epochs, alpha='asymmetric', eta='auto', per_word_topics=True, eval_every=None, minimum_probability=1e-8)
        
        self.old_model = LDA_model
        
        coherence_model_lda = CoherenceModel(model=LDA_model, texts=songs, dictionary=country_dict, coherence='u_mass')
        coherence_lda = coherence_model_lda.get_coherence()
        print('\nCoherence Score: ', coherence_lda)
        end = time.time()
        runtime = abs(start-end)
        print("Overall model training runtime is", runtime)
        doc_lda = LDA_model[Corpus]

        pyLDAvis.enable_notebook()
        lda_viz = gensimvis.prepare(LDA_model, Corpus, country_dict)
        self.corpus = Corpus
        self.topic_words = doc_lda
        self.visualize = lda_viz
        
    def scrape_more_lyrics(self, n=200, a=200):
        '''
        Scrapes more lyrics to the dataframe file.
        
        n (int): the desired number of lyrics you want to have in the dataframe
        a (int): the index of the song you want to start scraping from
        '''
        
        print ("length of self.present is", len(self.present))
        if n <= len(self.present):
            print(f"You already have {n} lyrics scraped.")
            return None
        
        #Memorize the last translated lyrics index. This will be used for further translation
        try:
            self.last_translated = self.present[-1]
        except AttributeError:
            self.last_translated = 0
        
        print("Current number of scraped songs is", len(self.present))

        missing_num = n - len(self.present)
        
        print(f"Need to scrape {missing_num} more lyrics")
        
        print(f"Start scraping {missing_num} songs starting from {a}...")

        #Scrape the missing number of lyrics
        additional_df = scrape_lyrics(f'data/song_charts/{self.country_name}.xlsx', num_songs=missing_num, a=a, create_csv=False)

        print("Scraping has ended.")
    
        #Unite old and new dataframes
        self.df = pd.concat([self.df, additional_df])
        self.additional_df = additional_df
        print("New length of the dataframe is", len(self.df))
        print("Recounting the indices...")
        self.count_indices()
        
        #Update the dataframe file
        self.df.to_csv(f'df_{self.country_name.lower()}.csv')

C:\Users\user\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
def process_country(country, num_songs=200, topic_num=10, force_preprocess=False, delete_repetitions=False):
    '''
    input should be lowercase
    type: str
    '''
    country = Country(country.capitalize(), num_songs=num_songs)
    country.count_indices()
    country.translate_all_lyrics()
    print(f"This is the proportion of languages in the songs of {country.country_name}:", country.langs)
    country.preprocess_lyrics(force_preprocess=force_preprocess, delete_repetitions=delete_repetitions)
    return country
    

C:\Users\user\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# start processing the chosen country

germany = process_country('germany', force_preprocess=True, delete_repetitions=False)

In [ ]:
germany.count_langs()

In [ ]:
# if you want to iterate through different number of clusters 

# iterate and save the results for each cluster number in the dictionary 

topics_results = defaultdict(dict)

for i in range(3, 8):
    germany.find_topics(filter_extremes=False, topics=i, epochs=200)
    topics_results[i] = {'visual': germany.visualize}
    name = germany.country_name

In [17]:
topics_results[3]['visual']

C:\Users\user\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.067463  0.104092       1        1  39.844719
0     -0.095399 -0.092832       2        1  37.176767
2      0.162862 -0.011259       3        1  22.978513, topic_info=         Term        Freq       Total Category  logprob  loglift
3280    bonez  106.000000  106.000000  Default  30.0000  30.0000
3461     gzuz   75.000000   75.000000  Default  29.0000  29.0000
110      love  398.000000  398.000000  Default  28.0000  28.0000
1069    taste   87.000000   87.000000  Default  27.0000  27.0000
3639  thunder   87.000000   87.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
89        way   19.210729  121.851526   Topic3  -5.9614  -0.3767
112     night   19.842345  213.084749   Topic3  -5.9291  -0.9033
125      time   18.831310  154.187657   Topic3  -5.9814  -0.6320
110      love   19.112433  398.352108   Topic3  -5.9666  -1.5664
48      lyric   18.033089  160.967192   Topic3  -6.0247  -0.7184

[219 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
1856      3  0.912154    110k
1857      3  0.932020    115k
1863      3  0.933101    145k
1870      3  0.903396    170k
1871      3  0.903396    175k
...     ...       ...     ...
1150      2  0.991273  wishin
2450      1  0.019464     wit
2450      3  0.973220     wit
4623      1  0.969823    woop
4428      1  0.944520  woulda

[311 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])

In [ ]:
# if you want to examine a specific number of clusters 

germany.find_topics(topics=5, filter_extremes=True, epochs=200)

In [ ]:
germany.visualize